In [82]:
import os

import pandas as pd
import numpy as np
from PIL import Image

from shared_astro_utils.matching_utils import match_galaxies_to_catalog_pandas

from galaxy_datasets.shared import label_metadata
from zoobot.shared import schemas

In [83]:
# final version of https://github.com/mwalmsley/curation-datasets/blob/main/notebooks/rename_gz2_catalog.ipynb

In [84]:
# hart = pd.read_csv('/run/media/walml/beta/galaxy_zoo/gz2/subjects/gz2_hart16.csv')
# len(hart)

In [85]:
# TODO I think is is a renaming of hart 2016
master_df = pd.read_parquet('/run/media/walml/beta/galaxy_zoo/gz2/subjects/image_master_catalog.parquet')
"""
original    209294
extra        21844
stripe82      8557
"""
len(master_df)



239695

In [86]:
save_loc = '/run/media/walml/beta/galaxy_zoo/gz2/subjects/gz2_catalog_with_modern_schema_inc_stripe82.parquet'

# original sample only
# save_loc = '/run/media/walml/beta/galaxy_zoo/gz2/subjects/gz2_catalog_with_modern_schema.parquet'
# master_df = master_df.query('sample == "original"')


len(master_df)

239695

In [87]:
renamer = {
    't01_smooth_or_features_a01_smooth': 'smooth-or-featured-gz2_smooth',
    't01_smooth_or_features_a02_features_or_disk': 'smooth-or-featured-gz2_featured-or-disk',
    't01_smooth_or_features_a03_star_or_artifact': 'smooth-or-featured-gz2_artifact',
    't02_edgeon_a04_yes': 'disk-edge-on-gz2_yes',
    't02_edgeon_a05_no': 'disk-edge-on-gz2_no',
    't03_bar_a06_bar': 'bar-gz2_yes',
    't03_bar_a07_no_bar': 'bar-gz2_no',
    't04_spiral_a08_spiral': 'has-spiral-arms-gz2_yes',
    't04_spiral_a09_no_spiral': 'has-spiral-arms-gz2_no',
    't05_bulge_prominence_a10_no_bulge': 'bulge-size-gz2_no',
    't05_bulge_prominence_a11_just_noticeable': 'bulge-size-gz2_just-noticeable',
    't05_bulge_prominence_a12_obvious': 'bulge-size-gz2_obvious',
    't05_bulge_prominence_a13_dominant': 'bulge-size-gz2_dominant',
    't06_odd_a14_yes': 'something-odd-gz2_yes',
    't06_odd_a15_no': 'something-odd-gz2_no',
    't07_rounded_a16_completely_round': 'how-rounded-gz2_round',
    't07_rounded_a17_in_between': 'how-rounded-gz2_in-between',
    't07_rounded_a18_cigar_shaped': 'how-rounded-gz2_cigar',
    't09_bulge_shape_a25_rounded': 'bulge-shape-gz2_round',
    't09_bulge_shape_a26_boxy': 'bulge-shape-gz2_boxy',
    't09_bulge_shape_a27_no_bulge': 'bulge-shape-gz2_no-bulge',
    't10_arms_winding_a28_tight': 'spiral-winding-gz2_tight',
    't10_arms_winding_a29_medium': 'spiral-winding-gz2_medium',
    't10_arms_winding_a30_loose': 'spiral-winding-gz2_loose',
    't11_arms_number_a31_1': 'spiral-arm-count-gz2_1',
    't11_arms_number_a32_2': 'spiral-arm-count-gz2_2',
    't11_arms_number_a33_3': 'spiral-arm-count-gz2_3',
    't11_arms_number_a34_4': 'spiral-arm-count-gz2_4',
    't11_arms_number_a36_more_than_4': 'spiral-arm-count-gz2_more-than-4',
    't11_arms_number_a37_cant_tell': 'spiral-arm-count-gz2_cant-tell'
}

renamer_with_count = dict([(key + '_count', value) for key, value in renamer.items()])

# copy the key id columns
df = master_df[['dr7objid', 'png_loc', 'sample', 'ra', 'dec']]

# copy all label columns using renamer
for key, value in renamer_with_count.items():
    df.loc[:, value] = master_df[key]

df


/nvme1/scratch/miniconda3/envs/zoobot/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,dr7objid,png_loc,sample,ra,dec,smooth-or-featured-gz2_smooth,smooth-or-featured-gz2_featured-or-disk,smooth-or-featured-gz2_artifact,disk-edge-on-gz2_yes,disk-edge-on-gz2_no,...,bulge-shape-gz2_no-bulge,spiral-winding-gz2_tight,spiral-winding-gz2_medium,spiral-winding-gz2_loose,spiral-arm-count-gz2_1,spiral-arm-count-gz2_2,spiral-arm-count-gz2_3,spiral-arm-count-gz2_4,spiral-arm-count-gz2_more-than-4,spiral-arm-count-gz2_cant-tell
0,587732591714893851,/raid/scratch/walml/galaxy_zoo/gz2/png/587732/...,original,179.042984,60.522518,0,42,3,2,40,...,0,27,12,1,0,3,1,2,18,16
1,588009368545984617,/raid/scratch/walml/galaxy_zoo/gz2/png/588009/...,original,135.084396,52.494240,1,41,0,0,41,...,0,35,4,2,0,4,2,1,21,13
2,587732484359913515,/raid/scratch/walml/galaxy_zoo/gz2/png/587732/...,original,183.371979,50.741508,28,5,3,0,5,...,0,0,0,0,0,0,0,0,0,0
3,587741723357282317,/raid/scratch/walml/galaxy_zoo/gz2/png/587741/...,original,186.251953,28.558598,1,27,0,1,26,...,1,16,8,1,0,2,6,5,6,6
4,587738410866966577,/raid/scratch/walml/galaxy_zoo/gz2/png/587738/...,original,161.086395,14.084465,33,8,2,1,7,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239690,587741490371625059,/raid/scratch/walml/galaxy_zoo/gz2/png/587741/...,extra,167.542641,28.991866,34,7,1,6,1,...,6,0,0,0,0,0,0,0,0,0
239691,587731174917669027,/raid/scratch/walml/galaxy_zoo/gz2/png/587731/...,stripe82,316.108837,1.214799,25,12,9,0,12,...,0,0,0,0,0,0,0,0,0,0
239692,587731512070177108,/raid/scratch/walml/galaxy_zoo/gz2/png/587731/...,stripe82,21.690211,-0.546427,36,7,5,0,7,...,0,1,1,0,0,0,0,0,0,2
239693,587731513145688256,/raid/scratch/walml/galaxy_zoo/gz2/png/587731/...,stripe82,25.747144,0.360079,25,14,9,0,14,...,0,1,0,0,1,0,0,0,0,0


In [88]:

# now the schema object will work, I can calculate fractions and totals

question_answer_pairs = label_metadata.gz2_ortho_pairs
dependencies = label_metadata.gz2_ortho_dependencies

# dependencies = {
#     'smooth-or-featured': None,  # always asked
#     'disk-edge-on': 'smooth-or-featured_featured-or-disk',
#     'has-spiral-arms': 'smooth-or-featured_featured-or-disk',
#     'bar': 'smooth-or-featured_featured-or-disk',
#     'bulge-size': 'smooth-or-featured_featured-or-disk',
#     'how-rounded': 'smooth-or-featured_smooth',
#     'bulge-shape': 'disk-edge-on_yes',  # gz2 only
#     'edge-on-bulge': 'disk-edge-on_yes',
#     'spiral-winding': 'has-spiral-arms_yes',
#     'spiral-arm-count': 'has-spiral-arms_yes',
#     'something-odd': None  # actually neglects the artifact branch
# }

schema = schemas.Schema(question_answer_pairs, dependencies)
for question in schema.questions:
    df[question.text + '_total-votes'] = df[[a.text for a in question.answers]].sum(axis=1)

    for answer in question.answers:
        df[answer.text + '_fraction'] = df[answer.text] / df[question.text + '_total-votes']





/nvme1/scratch/miniconda3/envs/zoobot/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/nvme1/scratch/miniconda3/envs/zoobot/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [89]:
# fix paths
# will eventually remove this prepended string
df['png_loc'] = df['png_loc'].str.replace('/raid/scratch/walml', '/run/media/walml/beta')
df['png_loc'][0]


/nvme1/scratch/miniconda3/envs/zoobot/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


'/run/media/walml/beta/galaxy_zoo/gz2/png/587732/587732591714893851.png'

In [90]:
# takes about 2 mins on lofar9
# df['png_exists'] = df['png_loc'].apply(os.path.isfile)

In [91]:
# df[~df['png_exists']].squeeze()['png_loc']
# # '/raid/scratch/walml/galaxy_zoo/gz2/png/587736/587736915146703138.png'
# # literally 1 image missing...

In [92]:


df = df[df['dr7objid'] != 587736915146703138]

# label_df['file_exists'].value_counts()

In [93]:
def get_jpg_loc(png_loc):
    return png_loc.replace('png', 'jpg').replace('/run/media/walml/beta/galaxy_zoo/gz2' , '/home/walml/repos/galaxy-datasets/roots/gz2').replace('/jpg/', '/images/')

In [94]:

def copy_jpg(png_loc):
    jpg_loc = get_jpg_loc(png_loc)
    jpg_dir = os.path.dirname(jpg_loc)
    if not os.path.isdir(jpg_dir):
        os.mkdir(jpg_dir)
    if not os.path.isfile(jpg_loc):
        # print(f'Making {jpg_loc}')
        Image.open(png_loc).save(jpg_loc)


In [95]:
# df['png_loc'].str.replace('png', 'jpg').str.replace('/run/media/walml/beta/galaxy_zoo/gz2' , '/home/walml/repos/galaxy-datasets/roots/gz2').iloc[0]

In [96]:
df['png_loc']

0         /run/media/walml/beta/galaxy_zoo/gz2/png/58773...
1         /run/media/walml/beta/galaxy_zoo/gz2/png/58800...
2         /run/media/walml/beta/galaxy_zoo/gz2/png/58773...
3         /run/media/walml/beta/galaxy_zoo/gz2/png/58774...
4         /run/media/walml/beta/galaxy_zoo/gz2/png/58773...
                                ...                        
239690    /run/media/walml/beta/galaxy_zoo/gz2/png/58774...
239691    /run/media/walml/beta/galaxy_zoo/gz2/png/58773...
239692    /run/media/walml/beta/galaxy_zoo/gz2/png/58773...
239693    /run/media/walml/beta/galaxy_zoo/gz2/png/58773...
239694    /run/media/walml/beta/galaxy_zoo/gz2/png/58801...
Name: png_loc, Length: 239694, dtype: object

In [97]:
copy_jpg(df['png_loc'][0])

In [98]:
# now make jpg version of every png
# takes about 40 mins from scratch

# now includes all GZ2 images (inc. stripe82) not just the original set

from PIL import Image

from multiprocessing import Pool

pool = Pool(20)

_ = [_ for x in pool.imap_unordered(copy_jpg, df['png_loc'])]


In [99]:
df['file_loc'] = df['png_loc'].apply(get_jpg_loc)
df['subfolder'] = df['file_loc'].apply(lambda x: os.path.basename(os.path.dirname(x)))

/nvme1/scratch/miniconda3/envs/zoobot/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/nvme1/scratch/miniconda3/envs/zoobot/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [100]:
# # now check all the jpgs made correctly
df['file_exists'] = df['file_loc'].apply(os.path.isfile)
df['file_exists'].value_counts()


/nvme1/scratch/miniconda3/envs/zoobot/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


True    239694
Name: file_exists, dtype: int64

In [101]:
from PIL import Image

def is_formatted_well(loc, careful=False):
    try:
        im =  Image.open(loc)
        if careful:
            mean = np.array(im).mean()  # avoid lazy loading (very slow now, though, )
        return im.size == (424, 424)
    except:
        return False

# check all jpgs open correctly - if not, delete the bad ones and remake
df['is_formatted_well'] = df['file_loc'].apply(is_formatted_well)
df['is_formatted_well'].value_counts()


/nvme1/scratch/miniconda3/envs/zoobot/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


True    239694
Name: is_formatted_well, dtype: int64

In [102]:
# now ready to be tarred and uploaded to dropbox

In [103]:
# TODO cross-match on sky with NSA here, for iauname

# first need to join to hard to get ra/dec
nsa = pd.read_parquet('/run/media/walml/beta/galaxy_zoo/decals/catalogs/nsa_v1_0_1_mag_cols.parquet', columns=['ra', 'dec', 'iauname'])

In [104]:


matched, unmatched = match_galaxies_to_catalog_pandas(df, nsa)
len(matched), len(unmatched)



/nvme1/scratch/walml/repos/shared-astro-utilities/shared_astro_utils/matching_utils.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  galaxies['best_match'] = best_match_catalog_index
/nvme1/scratch/walml/repos/shared-astro-utilities/shared_astro_utils/matching_utils.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  galaxies['sky_separation'] = sky_separation.to(units.arcsec).value


(218458, 20936)

In [105]:
df = pd.merge(df, matched[['iauname', 'dr7objid']], on='dr7objid', how='left')  # some iaunames will be nan

In [106]:

df.to_parquet(save_loc, index=False)